In [1]:
import numpy as np
import pandas as pd
import re

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from nltk.corpus import wordnet

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
# dataのload
data_sentence = pd.read_table('./Emobank-master/corpus/raw.tsv')
data_vad = pd.read_table('./Emobank-master/corpus/reader.tsv')
data_merged = pd.merge(data_sentence,data_vad,on='id')
print(len(data_sentence),len(data_vad),len(data_merged))

10358 10325 10142


In [3]:
ser = data_merged['sentence']

# 数字を全て0に
p_num = re.compile(r'[0-9]+')
ser = ser.apply(lambda x:p_num.sub("0",x))

# urlをremove 
p_url = re.compile(r'http[a-zA-Z0-9\!\#\$\%\&\'\*\+\-\.\^_\`\|\~\:]*')
ser = ser.apply(lambda x:p_url.sub(" ",x))

# replace .,! -> \s
ser = ser.apply(lambda x: x.replace("."," ").replace(","," ").replace("!"," ").replace("'","").replace("\"","").replace("“","").replace("”",""))

# lowerに
ser = ser.apply(lambda x:x.lower())

data_preprocessed = data_merged.assign(
    reg = ser
)


In [4]:
from nltk import word_tokenize
# sent = "This is my text, this is a nice way  to input text. I'm angry"
# word_tokenize(sent)
data_preprocessed = data_preprocessed.assign(
    words = data_preprocessed.apply(lambda x: len(word_tokenize(x['reg'])), axis=1)
)

In [5]:
# data_preprocessed_2 is '" -> space to '" -> blank ver 
data_preprocessed.to_csv('data_preprocessed_2.csv',encoding="utf-16")

In [6]:
data_preprocessed.head()

,id,sentence,Arousal,Dominance,Valence,sd.Arousal,sd.Dominance,sd.Valence,freq,reg,words
0,Acephalous-Cant-believe_4_47,I can't believe I wrote all that last year.,3.4,3.2,3.0,0.8,0.4,0.000000,5,i can t believe i wrote all that last year,10
1,Acephalous-Cant-believe_83_354,Because I've been grading all damn day and am ...,3.2,3.2,2.8,0.4,0.4,0.400000,5,because i ve been grading all damn day and am ...,49
2,Acephalous-Cant-believe_355_499,"However, when I started looking through my arc...",3.0,3.2,3.4,0.0,0.4,0.489898,5,however when i started looking through my arc...,26
3,Acephalous-Cant-believe_500_515,What do I mean?,3.0,3.0,3.0,0.0,0.0,0.000000,5,what do i mean?,5
4,Acephalous-Cant-believe_517_626,The posts I consider foundational to my curren...,3.0,3.0,3.0,0.0,0.0,0.000000,5,the posts i consider foundational to my curren...,18


In [19]:
col = ['Valence','Arousal','Dominance']

for name in col:
    rate1 = len(data_preprocessed[(data_preprocessed[name]>=2.8)&(data_preprocessed[name]<=3.2)])/len(data_preprocessed)
    vad_mask = ((data_preprocessed[name]>=2.8)&(data_preprocessed[name]<=3.2))
    data_1 = data_preprocessed[vad_mask]
    data_2 = data_preprocessed[~(vad_mask)]
    data_use,data_no_use = train_test_split(data_1, test_size=0.8)
    data_cut = pd.concat([data_use,data_2])
    rate2 = len(data_cut[(data_cut[name]>=2.8)&(data_cut[name]<=3.2)])/len(data_cut)
    print("{0} -> {1}".format(rate1,rate2))
    train, dev_test = train_test_split(data_cut,test_size=0.4)
    dev, test = train_test_split(dev_test,test_size=0.25)
    # train/dev/test = 6/3/1
    train = train.assign(data_type='train')
    dev = dev.assign(data_type='dev')
    test = test.assign(data_type='test')
    data_cut = pd.concat([train,dev,test])
    a=1
    b=5
    data_cut = data_cut.assign(
        Valence_reg = data_cut.apply(lambda x: 2*(x['Valence']-a)/(b-a)-1, axis=1),
        Arousal_reg = data_cut.apply(lambda x: 2*(x['Arousal']-a)/(b-a)-1, axis=1),
        Dominance_reg = data_cut.apply(lambda x: 2*(x['Dominance']-a)/(b-a)-1, axis=1)
    )
    # [-1,1]で正規化
    data_cut.to_csv('data_preprocessed_{0}.csv'.format(name),encoding='utf-16')

0.620193255768093 -> 0.2461839530332681
0.6566752119897457 -> 0.27669297881179894
0.7308223230132124 -> 0.35185185185185186


In [18]:
data_arousal = pd.read_csv('data_preprocessed_Arousal.csv',encoding='utf-16')
rate_a = len(data_arousal[(data_arousal['Arousal']>=2.8)&(data_arousal['Arousal']<=3.2)])/len(data_arousal)
rate_t = len(data_arousal[(data_arousal['data_type']=='train')])/len(data_arousal)

print(rate_a,rate_t)

0.27669297881179894 0.5999169090153719
